# Single-particle motion workbook

This notebook provides an investigation into single-particle motion within a laser.  Exact, analytical solutions of the particle motion are obtained from J. Yang, R. Craxton, and M. Haines, "Explicit general solutions to relativistic electron dynamics in plane-wave electromagnetic fields and simulations of ponderomotive acceleration." Plasma Phys. Contr. Fusion **53**, 125006 (2011).  This workbook can be used to explore the accuracy of particle-in-cell (PIC) simulations when simulating just a single-particle.  Parameters such as the laser intensity as well as initial phase can be varied to obtain different results.

Consider a laser with profile

$\vec{A}=\hat{x} A(\xi) \cos (k_0 \xi+\phi), \qquad \vec{E} = \hat{x}\left[ -\frac{dA}{d\xi} \cos (k_0 \xi+\phi) + A(\xi) k_0 \sin (k_0 \xi+\phi) \right], \qquad \vec{k}=\hat{z}k_0$.

If we initialize a particle in a laser with constant amplitude $A(\xi)=A_0$ and with zero initial velocity at $x=z=0$ and $\phi=\pi/2$, then we can express the longitudinal motion $z(t)$ as a drift plus some oscillation.  An explicit solution for this is given in Warren's notes as

$z(t) = v_d t + \frac{1}{k_0} \sum_{n=1}^{\infty} \frac{(-1)^{n}}{n!} J_n\left(\frac{n v_d}{c}\right) \sin\left(2n\Omega_0t\right)$,

where $v_d = \frac{c A_0^2}{4+A_0^2}$ and $\Omega_0=k_0 c\left( 1-\frac{v_d}{c} \right)$.  One can also obtain explicit time solutions for $x$, $p_z$, and $p_x$, which are not given here.  For the rest of this workbook, analytical solutions will be taken from Yang et al., which allow for general initial conditions of the particle and phase of the laser.

Below you can run OSIRIS simulations with your own parameters for the laser amplitude ($A_0$), initial phase deviation from $\pi/2$ (could be thought of as $\delta \phi$, or $\phi$ in the input deck below), and initial longitudinal proper velocity of the particle ($u_{z0}=\gamma v_{z0}$).  You will also be able to run with various field solvers and particle pushers.  The laser profile is flat over the middle half of the simulation, then tapers off to zero at the edges.  This is necessary to avoid unphysical results from boundary effects.

The two field solvers available are the Yee and Fei solvers.  The Yee solver is a standard among PIC codes, but has an imperfect dispersion relation for light waves in vacuum.  The Fei solver uses a 16-point stencil in the longitudinal direction to correct for dispersion errors in solving Maxwell's equations as well as for offset electric and magnetic fields in time.

The four particle pushers are Boris (standard), Vay (vay), conditional Vay (cond_vay), and Cary (cary).  The Boris push is standard for PIC codes.  The Vay pusher preserves the $\vec{E}\times\vec{B}$ velocity but is not volume-preserving, and is useful when particles attain very high energies (for large values of $A_0$).  The conditional Vay pusher uses the Boris push for particles with $\gamma<5$ and the Vay pusher otherwise.  The Cary pusher preserves the $\vec{E}\times\vec{B}$ velocity like the Vay pusher, but it is also volume-preserving, like the Boris push.

Several examples that have proved insightful include the following:

* Yee, $t_f=300$, $u_{z0}=0$, $a_0=5$, $\phi=-4$, try all pushers

OSIRIS simulations are done in normalized units. 

* Time:  $t' = t \omega_{p}$

* Frequency: $\omega' = \frac{\omega}{\omega_{p}}$ 

* Position: $\vec{x}' = \frac{\omega_{p}}{c} \vec{x}$  

* Momenta: $\vec{u}' = \frac{\vec{p}}{m_{e} c} = \frac{\vec{\gamma v}}{c} = \frac{\vec{u}} {c}$  

* Electric field: $\vec{E}' = e \frac{c / \omega_{p}}{m_{e} c^2} \vec{E}$  

* Magnetic field: $\vec{B}' = e \frac{c / \omega_{p}}{m_{e} c^2} \vec{B}$  


In [ ]:
# **********************************
# First Run This Cell to 
# import the necessary libraries
# **********************************
#
# Please run this cell before running anything, and run this cell again if you have restarted the 
# python kernel.
#
# This cell imports useful routines to diagnose the simulations here
#

import h5py
%matplotlib inline
import numpy as np
import sys, os
import osiris
import matplotlib.pyplot as plt
from single_particle_helper import single_particle_widget, newifile, plot_data
from ipywidgets import interact_manual,Layout,interact, FloatSlider
import ipywidgets as widgets
interact_calc=interact_manual.options(manual_name="Make New Input and Run")

### Simulation Sandbox --- Make your own simulation

In the widget below, enter these parameters and a name for the output. When the button is clicked, the function will make an input deck for OSIRIS.  Then execute OSIRIS in your terminal to run the simulation.  After the simulation has completed, execute the next cell, which will plot the data along with the exact analytical solution.

In [ ]:
## Hit <shift>+<return> to start the widget, which contains a interface to change OSIRIS parameters

single_particle_widget()

In [ ]:
## Hit <shift>+<return> to plot the data along with the theory

plot_data('single-part-1',xlim_max=None)